# Quadratically Constrained Quadratic Programs
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

- Introduce Quadratic Programs
    - Mathematical model (incl. constraints) + variable types<br>
    https://en.wikipedia.org/wiki/Quadratically_constrained_quadratic_program
    - How to build it with docplex (brief example)
    - How to build the same with the QuadraticProgram
- Introduce Converters <br>(except "to operator" and "to neg. val. oracle", those should be introduced with the algorithms)
    - Integer to Binary Converter
    - Penalize Equality Constraints
    - ...
- Example:
    - build a model in docplex with integer variables and constraints, cast to QUBO. 
    - introduce convenience converter "to_qubo"...

### Converters for `QuadraticProgram`
To solve an optimization prbolem with quantum computer, a corresponding Ising Hamiltonian for the optimization problem is needed to be created. To do so, firstly, it's necessary to convert it into with a specific form, more precisely Unconstrained Binary Optimization form. For that purpose, Qiskit has three converters. They are `InequalityToEqualityConverter`, `IntegerToBinaryConverter`  and `PenalizeLinearEqualityConstraints`.With these converters, optimization problems can be easily and automatically converted into one with a specific form to solve quantum computers. The following is a list of three converters. The details of each converter will be discussed later with examples.
- `InequalityToEqualityConverter`: converts inequality constraints into equality constraints with additional slack variables.
- `IntegerToBinaryConverter`: converts integer variables into binary variables and corresponding coefficients. 
- `PenalizeLinearEqualityConstraints`: convert equality constraints into additional terms of the object function.

#### `InequalityToEqualityConverter`
`InequalityToEqualityConverter` converts inequality constraints into equality constraints with additional slack variables to remove inequality constraints from `QuadraticProgram`. The upper bounds and the lower bounds of slack variables will be calculated from the difference between the left sides and the right sides of constraints. Signs of slack variables depend on symbols in constraints such as $\leq$ and $\geq$.

##### A usage example of `InequalityToEqualityConverter`
The following is a toy example of a maximization problem with an inequality constraint. Variable $x$ and $y$ are binary variables. variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\\
   & \text{subject to}
       & x+y+z \leq 5\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [8]:
op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z'], types='BBI', lb=[0, 0, 0], ub=[1, 1, 5])
op.objective.set_linear([('x', 2),('y', 1),('z', 1)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z'], val=[1, 1, 1])],
    senses=['L'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z <= 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 5
Binaries
 x  y 
Generals
 z 
End



Call `encode` method of `InequalityToEqualityConverter` to convert.

In [11]:
from qiskit.optimization.converters import InequalityToEqualityConverter
ineq2eq_conv = InequalityToEqualityConverter()
new_op = ineq2eq_conv.encode(op)
print(new_op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= Inf
 0 <= z <= 5
 0 <= xyz@int_slack <= 5
Binaries
 x  y 
Generals
 z  xyz@int_slack 
End



After converting, the inequality constraint is replaced with an equality constraint with an additional slack variable as the above. 

#### `IntegerToBinaryConverter`

`IntegerToBinaryConverter` converts integer variables into binary variables and coefficients to remove integer variables from `QuadraticProgram`. For converting, bounded-coefficient encoding proposed in arxiv:1706.01945 (Eq. (5)) is used. For more detail of the encoding method, please see the paper.

##### A usage example of `IntegerToBinaryConverter`
The following is a toy example of a maximization problem with an integer variable. The following example is also the output of the above `InequalityToEqualityConverter` example. Variable $x$, $y$ and xyz@int_slack are binary variables. variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\\
   & \text{subject to}
       & x+y+z +xyz@int\_slack= 5\\
       & & x, y, xyz@int\_slack \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [13]:
op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z','xyz@int_slack'], types='BBIB', lb=[0, 0, 0, 0], ub=[1, 1, 5, 1])
op.objective.set_linear([('x', 2),('y', 1),('z', 1)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z', 'xyz@int_slack'], val=[1, 1, 1, 1])],
    senses=['E'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 5
 0 <= xyz@int_slack <= 1
Binaries
 x  y  xyz@int_slack 
Generals
 z 
End



Call `encode` method of `IntegerToBinaryConverter` to convert.

In [14]:
from qiskit.optimization.converters import IntegerToBinaryConverter
int2bin_conv = IntegerToBinaryConverter()
new_op = int2bin_conv.encode(op)
print(new_op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z@0 + 2 z@1 + 2 z@2
Subject To
 xyz: x + y + z@0 + 2 z@1 + 2 z@2 + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z@0 <= 1
 0 <= z@1 <= Inf
 0 <= z@2 <= Inf
 0 <= xyz@int_slack <= 1
Binaries
 x  y  z@0  z@1  z@2  xyz@int_slack 
End



After converting, integer variables $z$ is replaced with three binary variables $z@0$, $z@1$ and $z@2$ with coefficients 1, 2 and 2, respectively as the above. 

#### `PenalizeLinearEqualityConstraints`

Common import?

In [3]:
from qiskit.optimization import OptimizationProblem
from cplex import SparsePair